In [14]:
import pandas as pd
from tqdm import tqdm
from imblearn.over_sampling import SMOTE
from scipy import stats

from sklearn.model_selection import cross_validate,train_test_split
from sklearn.metrics  import accuracy_score,make_scorer

import numpy as np
import warnings
import optuna
from optuna.pruners import SuccessiveHalvingPruner
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import make_pipeline, Pipeline
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from optuna.samplers import TPESampler
warnings.filterwarnings('ignore')
%matplotlib inline

https://www.kaggle.com/nguyenthanhtrung/random-forest-optuna-private-lb-0-5242
https://dacon.io/competitions/official/235840/codeshare/3794?page=1&dtype=recent
https://data-newbie.tistory.com/396

In [2]:
def zscore_df(df):
        
        data = df.iloc[:,1:-1].apply(stats.zscore)
        
        data['quality'] = df['quality']
        data['id'] = df['id']
        #data['type'] = df['type']
        
        for i in data.iloc[:,:-2].columns:
            data = data.loc[(data[i] > -2) & (data[i] < 2)]
       
        return data



In [3]:
train_path = '../train/train.csv'
test_path = '../test/test.csv'
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)


In [4]:
df = train_df

word_to_num = {'white' : 0, 'red' : 1}
df['type'] = df['type'].replace(word_to_num)

In [5]:
zscore_df = zscore_df(train_df)

In [5]:
X = df.drop(['id', 'quality'],axis=1)
y = df['quality']

In [ ]:
features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 
            'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 
            'pH', 'sulphates', 'alcohol', 'type']
target = 'quality'

In [ ]:
for train_index, valid_index in skf.split(train[features], df[target]):
    df_train = df.loc[train_index]
    df_valid = df.loc[valid_index]
    df_trains.append(df_train)
    df_valids.append(df_valid)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.4,random_state=42)

In [7]:

    
def objective(trial: optuna.trial.Trial):

    params = {
        'n_estimators' : trial.suggest_int('n_estimators', 0, 1000),
        'max_depth' : trial.suggest_int('max_depth', 1, 50),
        'min_samples_split' : trial.suggest_int('min_samples_split',1,150),
        'min_samples_leaf' : trial.suggest_int('min_samples_leaf',1,60)
    }
    smote_neighbors = trial.suggest_int('smoth_n_neighbors', 1, 10)
    sampler = SMOTE(random_state=42, k_neighbors=smote_neighbors)
    clf = RandomForestClassifier(random_state=42, **params)
    pipeline = make_pipeline(sampler, clf)
    scores = cross_validate(pipeline, X_train, 
                            y_train, verbose=1 , n_jobs=-1, 
                            scoring=make_scorer(accuracy_score), cv=4)
    return scores["test_score"].mean()

studies = optuna.create_study(direction='maximize', pruner=SuccessiveHalvingPruner())
studies.optimize(objective, n_trials=50)
    
print(f'===== Done fold =====')
print(studies.best_value)
print(studies.best_params)

[I 2021-12-06 07:33:07,002] A new study created in memory with name: no-name-04444a4c-0498-4fc2-9ba1-cba7dea4a159
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   4 out of   4 | elapsed:    3.8s finished
[I 2021-12-06 07:33:10,951] Trial 0 finished with value: 0.4205450711425407 and parameters: {'n_estimators': 68, 'max_depth': 50, 'min_samples_split': 86, 'min_samples_leaf': 40, 'smoth_n_neighbors': 4}. Best is trial 0 with value: 0.4205450711425407.
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   4 out of   4 | elapsed:    5.9s finished
[I 2021-12-06 07:33:16,944] Trial 1 finished with value: 0.41795710147397114 and parameters: {'n_estimators': 523, 'max_depth': 31, 'min_samples_split': 104, 'min_samples_leaf': 32, 'smoth_n_neighbors': 2}. Best is trial 0 with value: 0.4205450711425407.
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jo

NotFittedError: All estimators failed to fit

In [ ]:

smote_best_params = {'smoth_n_neighbors': 6}

sampler= SMOTE(random_state=42, k_neighbors=smote_best_params["smoth_n_neighbors"])
clf = RandomForestClassifier(random_state=42, n_estimators = 630,
                            max_depth = 30, min_samples_split = 6,
                            min_samples_leaf = 1)

pipeline = Pipeline([("sampler", sampler), ("clf", clf)])
pipeline.fit(X_train, y_train)
preds_valid = clf.predict(X_test.values)
acc = accuracy_score(y_test, preds_valid)
print('ACC:', acc)



In [31]:

test = pd.read_csv('../test/test.csv')
word_to_num = {'white' : 0, 'red' : 1}
test['type'] = test['type'].replace(word_to_num)
#test.drop('type',axis=1,inplace=True)
test =  test.apply(stats.zscore)
prediction = clf.predict(test.drop('id', axis=1))

submission = pd.read_csv('../sample_submission.csv')
submission['quality'] = prediction
submission.to_csv("../submission/rf_optuna.csv", index=False)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.4,random_state=42)
    
def objective(trial: optuna.trial.Trial):

    params = {
        'n_estimators' : trial.suggest_int('n_estimators', 0, 1000),
        'max_depth' : trial.suggest_int('max_depth', 4, 50),
        'min_child_wiehgt' : trial.suggest_int('min_child_weight',1,20),
        'scale_pos_weight' : trial.suggest_int('scale_pos_weight',1,100),
        'subsample' : trial.suggest_discrete_uniform('subsample',0.5,0.9,0.1),
        'colsample_bytree' : trial.suggest_discrete_uniform('colsample_bytree',0.5,0.9,0.1)
    }
    smote_neighbors = trial.suggest_int('smoth_n_neighbors', 5, 10)
    #sampler = SMOTE(random_state=42, k_neighbors=smote_neighbors)
    clf = XGBClassifier(random_state=42, **params)
    #pipeline = make_pipeline(sampler, clf)
    scores = cross_validate(clf, X_train, 
                            y_train, verbose=1 , n_jobs=-1, 
                            scoring=make_scorer(accuracy_score), cv=4)
    return scores["test_score"].mean()

studies = optuna.create_study(direction='maximize',sampler=TPESampler(seed=0), pruner=SuccessiveHalvingPruner())
studies.optimize(objective, n_trials=50)
    
print(f'===== Done fold =====')
print(studies.best_value)
print(studies.best_params)

[I 2021-12-06 08:24:46,145] A new study created in memory with name: no-name-8f542b7b-45af-4f69-abd6-e32204c1bb7f
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    6.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    6.5s finished
[I 2021-12-06 08:24:52,786] Trial 0 finished with value: 0.5753237624605947 and parameters: {'n_estimators': 549, 'max_depth': 37, 'min_child_weight': 13, 'scale_pos_weight': 55, 'subsample': 0.7, 'colsample_bytree': 0.8, 'smoth_n_neighbors': 7}. Best is trial 0 with value: 0.5753237624605947.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    7.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    7.9s finished
[I 2021-12-06 08:25:00,821] Trial 1 finished with value: 0.5619131805401721 and parameters: {'n_estimators': 892, 'max_depth': 49, 'min

[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    6.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    6.2s finished
[I 2021-12-06 08:26:14,341] Trial 15 finished with value: 0.5779106671210701 and parameters: {'n_estimators': 736, 'max_depth': 30, 'min_child_weight': 4, 'scale_pos_weight': 42, 'subsample': 0.6, 'colsample_bytree': 0.6, 'smoth_n_neighbors': 7}. Best is trial 15 with value: 0.5779106671210701.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    5.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    5.7s finished
[I 2021-12-06 08:26:20,168] Trial 16 finished with value: 0.5737848257646758 and parameters: {'n_estimators': 620, 'max_depth': 31, 'min_child_weight': 3, 'scale_pos_weight': 41, 'subsample': 0.8, 'colsample_bytree': 0.7, 'smoth_n_neighbors': 9}. Best is trial 15 with value: 0.5779106671210701.
[Parallel(n_jobs=-1)]: Using

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    3.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    3.6s finished
[I 2021-12-06 08:27:07,516] Trial 31 finished with value: 0.5825530374030842 and parameters: {'n_estimators': 281, 'max_depth': 50, 'min_child_weight': 2, 'scale_pos_weight': 17, 'subsample': 0.5, 'colsample_bytree': 0.8, 'smoth_n_neighbors': 9}. Best is trial 22 with value: 0.5851356820311834.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    4.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    4.3s finished
[I 2021-12-06 08:27:12,006] Trial 32 finished with value: 0.5861655448581409 and parameters: {'n_estimators': 411, 'max_depth': 50, 'min_child_weight': 3, 'scale_pos_weight': 11, 'subsample': 0.5, 'colsample_bytree': 0.8, 'smoth_n_neighbors': 9}. B

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    3.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    3.8s finished
[I 2021-12-06 08:28:12,213] Trial 47 finished with value: 0.5815157195194683 and parameters: {'n_estimators': 334, 'max_depth': 47, 'min_child_weight': 3, 'scale_pos_weight': 8, 'subsample': 0.6, 'colsample_bytree': 0.8, 'smoth_n_neighbors': 9}. Best is trial 43 with value: 0.5866831387918547.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.9s finished
[I 2021-12-06 08:28:15,220] Trial 48 finished with value: 0.5748210786401976 and parameters: {'n_estimators': 233, 'max_depth': 37, 'min_child_weight': 10, 'scale_pos_weight': 12, 'subsample': 0.5, 'colsample_bytree': 0.7, 'smoth_n_neighbors': 8}. B

===== Done fold =====
0.5866831387918547
{'n_estimators': 394, 'max_depth': 45, 'min_child_weight': 4, 'scale_pos_weight': 23, 'subsample': 0.5, 'colsample_bytree': 0.8, 'smoth_n_neighbors': 9}


In [ ]:
print(f'===== Done fold =====')
print(studies.best_value)
print(studies.best_params)

In [ ]:
clf_best_params = {'n_estimators': 778, 'max_depth': 23, 'min_child_weight': 14, 'scale_pos_weight': 84,
                   'subsample': 0.9, 'colsample_bytree': 0.6}
smote_best_params = {'smoth_n_neighbors': 8}

sampler= SMOTE(random_state=42, k_neighbors=smote_best_params["smoth_n_neighbors"])
clf = XGBClassifier(random_state=42, params = clf_best_params)

pipeline = Pipeline([("sampler", sampler), ("clf", clf)])
pipeline.fit(X_train, y_train)
preds_valid = clf.predict(X_test.values)
acc = accuracy_score(y_test, preds_valid)
print('ACC:', acc)



In [34]:

test = pd.read_csv('../test/test.csv')
word_to_num = {'white' : 0, 'red' : 1}
test['type'] = test['type'].replace(word_to_num)
#test.drop('type',axis=1,inplace=True)
test =  test.apply(stats.zscore)
prediction = clf.predict(test.drop('id', axis=1))

submission = pd.read_csv('../sample_submission.csv')
submission['quality'] = prediction
submission.to_csv("../submission/xgb_optuna.csv", index=False)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.4,random_state=42)
    
def objective(trial: optuna.trial.Trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024, step=1, log=True), 
        'max_depth': trial.suggest_int('max_depth', 1, 10, step=1, log=False), 
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1, log=True), 
        'n_estimators': trial.suggest_int('n_estimators', 8, 1024, step=1, log=True), 
        'objective': 'multiclass', 
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50, step=1, log=False), 
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0), 
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 10.0),
        'random_state': 0}
    
        
    smote_neighbors = trial.suggest_int('smoth_n_neighbors', 5, 10)
    sampler = SMOTE(random_state=42, k_neighbors=smote_neighbors)
    #sampler=TPESampler(seed=0)
    clf = LGBMClassifier( **params)
    pipeline = make_pipeline(sampler, clf)
    scores = cross_validate(clf, X_train, 
                            y_train, verbose=1 , n_jobs=-1, 
                            scoring=make_scorer(accuracy_score), cv=4)
    return scores["test_score"].mean()

studies = optuna.create_study(direction='maximize',sampler=TPESampler(seed=0), pruner=SuccessiveHalvingPruner())
studies.optimize(objective, n_trials=50)
    
print(f'===== Done fold  =====')
print(studies.best_value)
print(studies.best_params)

[I 2021-12-06 08:32:50,582] A new study created in memory with name: no-name-477cb2af-45c7-4d5b-8782-52b9de323a73
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    1.6s finished
[I 2021-12-06 08:32:52,273] Trial 0 finished with value: 0.534591462895118 and parameters: {'num_leaves': 54, 'max_depth': 8, 'learning_rate': 0.006431172050131994, 'n_estimators': 109, 'class_weight': None, 'min_child_samples': 27, 'subsample': 0.9675319002346239, 'colsample_bytree': 0.9890988281503088, 'reg_alpha': 0.3834415188257777, 'reg_lambda': 7.917250380826646, 'smoth_n_neighbors': 8}. Best is trial 0 with value: 0.534591462895118.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.5s finished
[I 2021-12-06 08:33:08,884] Trial 12 finished with value: 0.5830663713044218 and parameters: {'num_leaves': 223, 'max_depth': 6, 'learning_rate': 0.08007479721255774, 'n_estimators': 213, 'class_weight': None, 'min_child_samples': 10, 'subsample': 0.7864720828594255, 'colsample_bytree': 0.8953941626047761, 'reg_alpha': 0.9614271441409401, 'reg_lambda': 2.4739302800417553, 'smoth_n_neighbors': 5}. Best is trial 12 with value: 0.5830663713044218.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    1.9s finished
[I 2021-12-06 08:33:10,958] Trial 13 finished with value: 0.42930902274857285 and parameters: {

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   16.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   16.9s finished
[I 2021-12-06 08:34:58,931] Trial 24 finished with value: 0.5887503195024282 and parameters: {'num_leaves': 34, 'max_depth': 8, 'learning_rate': 0.0068152812398199845, 'n_estimators': 983, 'class_weight': None, 'min_child_samples': 24, 'subsample': 0.8845411238685706, 'colsample_bytree': 0.7664392024368014, 'reg_alpha': 0.022434445391960184, 'reg_lambda': 5.545781750419172, 'smoth_n_neighbors': 6}. Best is trial 24 with value: 0.5887503195024282.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    8.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    8.0s finished
[I 2021-12-06 08:35:07,109] Trial 25 finished with value: 0.565545923149016 and parameters: {

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   11.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   11.6s finished
[I 2021-12-06 08:37:46,617] Trial 36 finished with value: 0.5737858907727699 and parameters: {'num_leaves': 115, 'max_depth': 9, 'learning_rate': 0.00473301940250486, 'n_estimators': 731, 'class_weight': None, 'min_child_samples': 27, 'subsample': 0.9326967337831976, 'colsample_bytree': 0.7982816323440375, 'reg_alpha': 0.35452078380712937, 'reg_lambda': 4.663615916582325, 'smoth_n_neighbors': 8}. Best is trial 24 with value: 0.5887503195024282.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   29.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   29.6s finished
[I 2021-12-06 08:38:16,338] Trial 37 finished with value: 0.5866799437675726 and parameters: {'

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   11.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   11.8s finished
[I 2021-12-06 08:39:53,528] Trial 48 finished with value: 0.5660518019936951 and parameters: {'num_leaves': 92, 'max_depth': 9, 'learning_rate': 0.0037966369921813834, 'n_estimators': 545, 'class_weight': None, 'min_child_samples': 28, 'subsample': 0.8185700919094783, 'colsample_bytree': 0.868017117614881, 'reg_alpha': 0.45554432395541383, 'reg_lambda': 6.554892934280573, 'smoth_n_neighbors': 8}. Best is trial 24 with value: 0.5887503195024282.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    1.5s finished
[I 2021-12-06 08:39:55,229] Trial 49 finished with value: 0.5583145181903383 and parameters: {'

===== Done fold  =====
0.5887503195024282
{'num_leaves': 34, 'max_depth': 8, 'learning_rate': 0.0068152812398199845, 'n_estimators': 983, 'class_weight': None, 'min_child_samples': 24, 'subsample': 0.8845411238685706, 'colsample_bytree': 0.7664392024368014, 'reg_alpha': 0.022434445391960184, 'reg_lambda': 5.545781750419172, 'smoth_n_neighbors': 6}


In [ ]:
clf_best_params = {'n_estimators': 778, 'max_depth': 23, 'min_child_weight': 14, 'scale_pos_weight': 84,
                   'subsample': 0.9, 'colsample_bytree': 0.6}
smote_best_params = {'smoth_n_neighbors': 8}

sampler= SMOTE(random_state=42, k_neighbors=smote_best_params["smoth_n_neighbors"])
clf = LGBMClassifier(random_state=42, params = clf_best_params)

pipeline = Pipeline([("sampler", sampler), ("clf", clf)])
pipeline.fit(X_train, y_train)
preds_valid = clf.predict(X_test.values)
acc = accuracy_score(y_test, preds_valid)
print('ACC:', acc)



In [ ]:

test = pd.read_csv('../test/test.csv')
word_to_num = {'white' : 0, 'red' : 1}
test['type'] = test['type'].replace(word_to_num)
#test.drop('type',axis=1,inplace=True)
test =  test.apply(stats.zscore)
prediction = clf.predict(test.drop('id', axis=1))

submission = pd.read_csv('../sample_submission.csv')
submission['quality'] = prediction
submission.to_csv("../submission/xgb_optuna.csv", index=False)

In [ ]:
sampling : SMOTE -> optuna.TPEsampler 